# <center>__Assignment 5__</center>

In [1]:
"""load the required packages"""

import numpy as np
import scipy.stats as stats
import re
import string
import pandas as pd
from nltk.corpus import stopwords
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import CategoricalNB, GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

## Question 1

### a)	Upload the income_evaluation_cat.csv provided on canvas.  The features in this data include workclass, education, race, and gender. The output variable is income and contains two categorical values (<=50k or >50k) indicating whether the income of an individual is less than/equal to $50,000 or greater than $50,000 respectively. Print the unique values of each variable in this data. 

In [10]:
"""load income_evaluation_cat.csv"""

income_evaluation_cat = pd.read_csv('income_evaluation_cat.csv')
# print unique values in each column
for col in income_evaluation_cat.columns:
    print(col)
    print(income_evaluation_cat[col].unique())

 workclass
[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
 education
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
 race
[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
gender
[' Male' ' Female']
 income
[' <=50K' ' >50K']


### b)	You will implement Naïve Bayes from scratch using Bayes’ rule. You can do your calculations in Python, but you would not use the sklearn package. Suppose that all the income_evaluation_cat.csv data you uploaded is the training data , classify a test instance, X = [“Private”, “Bachelors”, “White”, “Female”] into the class income<=50 or income>50k. You need to compute the posterior probabilities P(income<=50/X) and P(income>50k/X), then print the class with the greater posterior probability as the predicted class. You don’t need to define a function, but you could if you want. 

In [11]:
"""clean the text  data in each column"""

# strip the whitespace from the column names in the data
income_evaluation_cat.columns = income_evaluation_cat.columns.str.strip()
# get column names from the datarame
colnames = income_evaluation_cat.columns
# strip whitespace from each attribute value
for col in colnames:
    income_evaluation_cat[col] = income_evaluation_cat[col].str.strip()

In [12]:
"""define Naive Bays classifier function"""

def naive_bays_classifier(data, target, new_record):

    # instantiate a list to store probabilities
    probs_less_50K = []
    probs_greater_50K = []
    # get record attributes form the test set
    attributes = data.columns.tolist()
    # compute the probability of observing the new_record attributes int the data
    for i in range(len(data.columns.tolist())-2):
        attribute = new_record[i]
        # compute the probability of observing the attribute given income <=50K
        less_50_k_prob = data[(data[target] == '<=50K') & (data[attributes[i]] == attribute)].shape[0]\
                              / data.shape[0]
        # append the probability for the attribute
        probs_less_50K.append(less_50_k_prob)
        # compute the probability of observing the attribute given income >50K
        greater_50_k_prob = data[(data[target] == '>50K') & (data[attributes[i]] == attribute)].shape[0]\
                              / data.shape[0]
        # append the probability for the attribute
        probs_greater_50K.append(greater_50_k_prob)

    # take product of y_value probability and likelhood of y_value/attribute combination
    less_50 = less_50_k_prob*np.prod(np.array(probs_less_50K))
    greater_50 = greater_50_k_prob*np.prod(np.array(np.prod(probs_greater_50K)))

    # prediction is max of (probability of target)*(likelihood of y_value/attribute combination)
    if less_50 > greater_50:
        return '<=50'
    else:
        return '>50'
    
# make predcition
predicted = naive_bays_classifier(income_evaluation_cat, 'income',
                                  ['Private', 'Bachelors', 'White', 'Female'])
# print prediction
predicted

'<=50'

### c)	Preprocess or transform the features in the income_evaluation_cat.csv data using an appropriate scaler in sklearn. You don’t need to transform the output variable; it should still work fine in a text format.

In [13]:
"""
apply sklearn ordinal encoder to the data to allow fitting
CategoricalNB estimator
"""

# get the columns
cols = income_evaluation_cat.iloc[:, :-1].columns
# instantiate ordinal encoder and fit_transform the data
income_evaluation_cat[cols] = OrdinalEncoder().fit_transform(income_evaluation_cat[cols])

### d)	Randomly split the transformed input data and the output data into X_train, y_train, X_test and y_test using tools in sklearn.

In [14]:
"""us sklearn shuffle split to randomly split input and output data"""

# separate the features and target
X = income_evaluation_cat.iloc[:, :-1]
y= income_evaluation_cat.iloc[:, -1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, shuffle=True,
                                                    random_state=42)

### e)	Use an appropriate Naïve Bayes constructor in sklearn to construct and fit a Naïve Bayes model on the training data, then use the model to compute the accuracy score of the training and test set.

In [15]:
"""construct categoricalNB estimator then fit to the data and score"""

# construct the estimator
cnb = CategoricalNB()
# fit estimator to training set
cnb.fit(X_train, y_train.to_numpy().flatten())

CategoricalNB()

In [16]:
"""compute accuracy score on the training set"""

# score against the training set
cnb.score(X_train, y_train)

0.7827746577746578

In [17]:
"""compute accuracy score on the test set"""

# score against the training set
cnb.score(X_test, y_test)

0.7879005015866517

## Question 2 

### a)	Upload the income_evaluation_continuous.csv data provided on canvas.  The features in this data include age, education_num, and hours_per_week. The output variable is income and contains two categorical values (<=50k or >50k) indicating whether the income of an individual is less than/equal to $50,000 or greater than $50,000 respectively. Compute the mean and standard deviation of each input variable such that the results are presented on the same table or data frame. You can call the .apply() function on the pandas DataFrame.

In [18]:
"""import the data and print the data head"""

income_evaluation_continuous = pd.read_csv('income_evaluation_continuous.csv')
income_evaluation_continuous.head()

,age,education_num,hours_per_week,income
0,39,13,40,<=50K
1,50,13,13,<=50K
2,38,9,40,<=50K
3,53,7,40,<=50K
4,28,13,40,<=50K


In [19]:
"""compute the mean and standard deviation of each input variable"""

income_evaluation_continuous.iloc[:, :-1].agg(func=['mean', 'std'])

,age,education_num,hours_per_week
mean,38.581647,10.080679,40.437456
std,13.640433,2.572720,12.347429


In [20]:
"""trim whitespace from column names"""

income_evaluation_continuous.columns = income_evaluation_continuous.columns.str.strip()

### b)	You will implement Naïve Bayes from scratch using Bayes’ rule. Assume that all the features or input variables follow a normal distribution. You can do your calculations in Python, but you would not use the sklearn package. You can use the density function inside the stats module in the SciPy package. Given that all the income_evaluation_continuous.csv data you uploaded is the training data , classify a test instance, X = [30, 10, 45], into the class income<=50 or income>50k. You need to compute the posterior probabilities P(income<=50/X) and P(income>50k/X), then print the class with the greater posterior probability as the predicted class. You don’t need to define a function, but you could if you want. 

In [21]:
"""implement Gaussian Naive Bayes classifier"""

def gaussian_nb(data, target, new_record):

    # get the mean of input features by class
    mean = data.groupby(target).mean()
    mean_less_50K = mean.iloc[0]
    mean_greater_50K = mean.iloc[1]
    # get standard deviation of input features
    std = data.groupby(target).std()
    std_less_50K = std.iloc[0]
    std_greater_50K = std.iloc[1]
    # compute the probability density of features in new_record for each class
    density_less_50K = stats.norm(mean_less_50K, std_less_50K).pdf(new_record)
    density_greater_50K = stats.norm(mean_greater_50K, std_greater_50K).pdf(new_record)
    # get the probabiity of each class label in the data
    p_y = data[target].value_counts()/sum(data[target].value_counts())
    y_less_50K = p_y[0]
    y_greater_50K = p_y[1]
    # compute the posterior probability of each class
    posterior_less_50K = y_less_50K*np.prod(density_less_50K)
    posterior_greater_50K = y_greater_50K*np.prod(density_greater_50K)
    # return the class prediction
    if posterior_less_50K > posterior_greater_50K:
        return "<=50K"
    else:
        return ">50K"

# create a new record then make predictions
new_record = [30, 10, 45]
gaussian_nb(income_evaluation_continuous, 'income', new_record)

'<=50K'

### c)	Preprocess or transform the features in the income_evaluation_cont.csv data using an appropriate scaler in sklearn. You don’t need to transform the output variable; it should still work fine in a text format.

In [22]:
"""transform numeric features in the data using standard scaler"""

# get input features in data
inputs = income_evaluation_continuous.iloc[:, :-1].columns
# instantiate standard scaler and fit_transform the data
sc = StandardScaler()
income_evaluation_continuous[inputs] = sc.fit_transform(income_evaluation_continuous[inputs])

### d)	Randomly split the input and output data into X_train, y_train, X_test and y_test using tools in sklearn.

In [23]:
"""randomly split the data into training and test sets"""

X = income_evaluation_continuous.iloc[:, :-1]
y= income_evaluation_continuous.iloc[:, -1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, shuffle=True, random_state=42)

### e)	Use an appropriate Naïve Bayes constructor in sklearn to construct and fit a Naïve Bayes model on the training data, then use the model to compute the accuracy score of the training and test set.

In [24]:
"""implement GaussianNB estimator, sklearn"""

gnb = GaussianNB()
gnb.fit(X_train, y_train.to_numpy().flatten())

GaussianNB()

In [25]:
"""compute the accuracy score on the training set"""

gnb.score(X_train, y_train)

0.7993594243594243

In [26]:
"""compute the accuracy score on the test set"""

gnb.score(X_test, y_test)

0.7988535162247927

## Question 3

### a)	Upload the True.csv data provided on canvas into Python. You will create a new data frame by selecting the “title” and “text” columns, then, adding a new column called “news_type” where all the values on this new column are “True”. So, your new data frame should have three columns; “title”, “text” and “news_type”.

In [27]:
"""upload the data, keep necessary columns, add new column news_type"""

# upload the data 
true = pd.read_csv('True.csv', usecols=['title', 'text'])
true['news_type'] = "True"

### b)	Upload the Fake.csv data provided on canvas into Python. You will create a new data frame by selecting the “title” and “text” columns, then, adding a new column called “news_type” where all the values on this new column are “Fake”. So, your new data frame should have three columns; “title”, “text” and “news_type”.

In [29]:
"""upload the data, keep necessary columns, add new column news_type"""

# upload the data 
fake = pd.read_csv('Fake.csv', usecols=['title', 'text'])
fake['news_type'] = "False"

### c)	Merge the data frame in a) and b) so that one of the data frames is stacked vertically on top of the other. Combine the text in the “title” and “text” columns of the merged data frame into another column called “news”. Drop the “title” and “text” columns so that your final data frame is has only two columns, “news” and “news_type”. Print the first five rows of your final data frame. 

In [30]:
"""
stack the data, combine the text in the title and text columns,
then drop the title and text columns
"""

# stack the data
news = pd.concat([true, fake], ignore_index=True)
# insert the new column
news.insert(loc=0, column='news', value=news.apply(lambda x: x['title'] + " " + x['text'], axis=1))
# drop the unnecessary columns
news.drop(columns=['title', 'text'], inplace=True)
# print the first 5 rows of data
news.head(5)

,news,news_type
0,"As U.S. budget fight looms, Republicans flip t...",True
1,U.S. military to accept transgender recruits o...,True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,True
3,FBI Russia probe helped by Australian diplomat...,True
4,Trump wants Postal Service to charge 'much mor...,True


### d)	Preprocess your data by cleaning the textual data in the “news” column and removing the stop words, special characters, punctuations, etc especially at the beginning and end of each word. You can display any messy news text before you clean the data, then display the messy news text again after cleaning the data to see if your data cleaning worked well. Also, drop instances where the news text is less than 50 words.

In [31]:
"""define a functiion to clean text data"""

def clean_text(text):

    # remove all non-alphanumeric characters while preserving spaces
    text = re.sub(r'[^a-zA-Z0-9 //g]', '', text)
    # get text as a list of words
    text = text.split()
    # remove all stop words
    stop_words = set(stopwords.words("english"))
    cleaner_text = [word.lower() for word in text if word.lower() not in stop_words]
    # check the length of cleaner text, if len(cleaner_text < 50) return false
    if len(cleaner_text) >= 50:
        return " ".join(cleaner_text)
    else:
        return None

In [32]:
"""print some text prior to cleaning"""

news['news'][0:51]

0     As U.S. budget fight looms, Republicans flip t...
1     U.S. military to accept transgender recruits o...
2     Senior U.S. Republican senator: 'Let Mr. Muell...
3     FBI Russia probe helped by Australian diplomat...
4     Trump wants Postal Service to charge 'much mor...
5     White House, Congress prepare for talks on spe...
6     Trump says Russia probe will be fair, but time...
7     Factbox: Trump on Twitter (Dec 29) - Approval ...
8     Trump on Twitter (Dec 28) - Global Warming The...
9     Alabama official to certify Senator-elect Jone...
10    Jones certified U.S. Senate winner despite Moo...
11    New York governor questions the constitutional...
12    Factbox: Trump on Twitter (Dec 28) - Vanity Fa...
13    Trump on Twitter (Dec 27) - Trump, Iraq, Syria...
14    Man says he delivered manure to Mnuchin to pro...
15    Virginia officials postpone lottery drawing to...
16    U.S. lawmakers question businessman at 2016 Tr...
17    Trump on Twitter (Dec 26) - Hillary Clinto

In [33]:
"""apply the clean_text function to the data"""

news['news'] = news['news'].apply(clean_text)

In [34]:
"""print some text following cleaning"""

news['news'][0:51]

0     us budget fight looms republicans flip fiscal ...
1     us military accept transgender recruits monday...
2     senior us republican senator let mr mueller jo...
3     fbi russia probe helped australian diplomat ti...
4     trump wants postal service charge much amazon ...
5     white house congress prepare talks spending im...
6     trump says russia probe fair timeline unclear ...
7     factbox trump twitter dec 29 approval rating a...
8     trump twitter dec 28 global warming following ...
9     alabama official certify senatorelect jones to...
10    jones certified us senate winner despite moore...
11    new york governor questions constitutionality ...
12    factbox trump twitter dec 28 vanity fair hilla...
13    trump twitter dec 27 trump iraq syria followin...
14    man says delivered manure mnuchin protest new ...
15    virginia officials postpone lottery drawing de...
16    us lawmakers question businessman 2016 trump t...
17    trump twitter dec 26 hillary clinton tax c

In [35]:
"""
check for null values returned by clean_text function
these represent text shorter than 50 words and should be dropped
"""

news['news'].isnull().sum()

3250

In [36]:
"""drop rows with null values"""

news.dropna(axis=0,inplace=True)
news.isnull().sum()

news         0
news_type    0
dtype: int64

### e)	Transform the input text data into feature vectors where the entries of the feature vectors are term-frequency-inverse-document-frequency. Use the TfidfVectorizer() in sklearn. 

In [37]:
"""get document feature vectors using sklearn"""

# instantiate TfidfVectorizer sklearn
tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', min_df=10, max_features=None)
# get document vectors
document_vectors = tfidf.fit_transform(news['news'])
document_vectors.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### f)	Split the feature vectors and the output variable into into X_train, y_train, X_test and y_test, you can let the test set be 30% of the entire data.

In [38]:
"""create the training and test sets"""

# get input feature vectors and target output
X = news.iloc[:, :-1]
y = news.iloc[:, -1:] 
# create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, shuffle=True, random_state=42)
# instantiate TfidfVectorizer sklearn
tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', min_df=10, max_features=None)
# get training and test document vectors
X_train_tfidf = tfidf.fit_transform(X_train['news'])
X_test_tfidf = tfidf.transform(X_test['news'])

### g)	Fit an appropriate Naïve Bayes model and compute the training and test accuracy of the model. Is there overfitting?

In [39]:
"""fit a multinomial Naive Bayes estimator sklearn"""

mnb = MultinomialNB()
mnb.fit(X_train_tfidf, y_train.to_numpy().flatten())

MultinomialNB()

In [40]:
"""compute accuracy score for training set"""

mnb.score(X_train_tfidf, y_train)

0.9630226734812883

In [41]:
"""compute accuracy score for test set"""

mnb.score(X_test_tfidf, y_test)

0.9544617847138855

The Multinomial Naive Bayes Model performed well on both the training and test sets, with nearly equivalent accuracy scores. Based on these results there does not appear to be a problem with overfitting.

### h)	Fit a Naïve Bayes using cross validation and print the average cross validation score as well as the standard deviation of the cross-validation scores. 

In [42]:
"""
fit the mulitnomial Naive Bayes Mmdel using cross validation
print the mean and standard deviation of cross-validation scores
"""

# get cross-validation scores
cross_val_scores = cross_val_score(estimator=MultinomialNB(), X=X_train_tfidf,
                                   y=y_train.to_numpy().flatten(), cv=3)
# print mean and standard deviation of scores
print("The average cross-validation accuracy score is: ", cross_val_scores.mean())
print("The standard deviation of cross-validation scores is: ", cross_val_scores.std())

The average cross-validation accuracy score is:  0.9546873720500213
The standard deviation of cross-validation scores is:  0.0015806293159337996


### i)	Select some hypermeters of your choice and tune using the grid search cross validation. Use some other hyperparameters than those used in class examples. 

In [43]:
"""isntantiate sklearn pipeline and GridSearchCV"""

# create the pipeline
pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', lowercase=False)), 
                    ('mnb', MultinomialNB())])
# specify param grid for tfidf and model
params = [{'tfidf__sublinear_tf': [True, False], 'tfidf__norm': ['l1', 'l2']}]
# instantiate and fit grid
grid_search = GridSearchCV(estimator=pipeline, param_grid=params, cv=3)
grid_search.fit(X_train['news'], y_train.to_numpy().flatten())

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(lowercase=False,
                                                        stop_words='english')),
                                       ('mnb', MultinomialNB())]),
             param_grid=[{'tfidf__norm': ['l1', 'l2'],
                          'tfidf__sublinear_tf': [True, False]}])

In [44]:
"""print the best parameters"""

grid_search.best_params_

{'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}

In [45]:
"""print accuracy score on training data"""

grid_search.score(X_train['news'], y_train.to_numpy().flatten())

0.9577401982643295

In [46]:
"""print accuracy score on test data"""

grid_search.score(X_test['news'], y_test.to_numpy().flatten())

0.9471788715486195

## 4) Mini Project

Find some text data of your own choice, it could be labelled tweets, etc. 
Your dataset should have at least 200 instances, and if there are several columns of text, you can choose to merge the text columns into a single text column. Each text instance should have at least 60 words.
 
Clean the data, split the data, transform the data to a representation suitable for your algorithm, build your model and evaluate the model. Tune some parameters of interest and write a short report about what problem your mini project is trying to address, the description of your data, the choice of algorithm used, the performance of your algorithm, overfitting, the choice of hyperparameters tunned, then your recommendation or conclusion (imagine you were trying to recommend this algorithm to a stakeholder, and you need this report to include important and persuasive elements). Your report could be in one or two paragraphs and should include relevant code and output at the end. 


## Amazon Food Reviews

In [48]:
"""read in the reviews data"""

reviews = pd.read_csv('reviews.csv', usecols=['Text', 'Score', 'Summary'], nrows=25000)
reviews.head()

,Score,Summary,Text
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,Great taffy,Great taffy at a great price. There was a wid...


In [49]:
"""combine the Summary and Text Columns then drop original"""

reviews.insert(loc=0, column='Review', value=reviews.apply(lambda x: x['Summary'] + " " + x['Text'], axis=1))
reviews.drop(columns=['Summary', 'Text'], inplace=True)

In [50]:
"""create new target feature "review_sentiment" """

reviews.insert(loc=2, column='review_sentiment', value=reviews.apply(lambda x: 'positive' if (x['Score'] > 3) else 'negative', axis=1))
reviews.drop(columns=['Score'], inplace=True)

In [51]:
"""define a functiion to clean reviews"""

def clean_reviews(text):

    # remove all non-alphanumeric characters while preserving spaces, @, and #
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # get text as a list of words
    text = text.split()
    # remove all stop words
    stop_words = set(stopwords.words("english"))
    cleaner_text = [word.lower() for word in text if word.lower() not in stop_words]
     # check the length of cleaner text, if len(cleaner_text < 60) return false
    if len(cleaner_text) >= 60:
        return " ".join(cleaner_text)
    else:
        return None

In [52]:
"print some reviews prior to cleaning"

reviews['Review'][0:51]

0     Good Quality Dog Food I have bought several of...
1     Not as Advertised Product arrived labeled as J...
2     "Delight" says it all This is a confection tha...
3     Cough Medicine If you are looking for the secr...
4     Great taffy Great taffy at a great price.  The...
5     Nice Taffy I got a wild hair for taffy and ord...
6     Great!  Just as good as the expensive brands! ...
7     Wonderful, tasty taffy This taffy is so good. ...
8     Yay Barley Right now I'm mostly just sprouting...
9     Healthy Dog Food This is a very healthy dog fo...
10    The Best Hot Sauce in the World I don't know i...
11    My cats LOVE this "diet" food better than thei...
12    My Cats Are Not Fans of the New Food My cats h...
13    fresh and greasy! good flavor! these came secu...
14    Strawberry Twizzlers - Yummy The Strawberry Tw...
15    Lots of twizzlers, just what you expect. My da...
16    poor taste I love eating them and they are goo...
17    Love it! I am very satisfied with my Twizz

In [53]:
"""clean reviews text"""

reviews['Review'] = reviews['Review'].apply(clean_reviews)

In [54]:
"print some reviews following cleaning"

reviews['Review'][0:51]

0                                                  None
1                                                  None
2                                                  None
3                                                  None
4                                                  None
5                                                  None
6                                                  None
7                                                  None
8                                                  None
9                                                  None
10    best hot sauce world dont know cactus tequila ...
11                                                 None
12                                                 None
13                                                 None
14                                                 None
15                                                 None
16                                                 None
17                                              

In [55]:
"""
check for null values returned by clean_tweet function
these represent text shorter than 60 words and should be dropped
"""

reviews['Review'].isnull().sum()

19699

In [56]:
"""drop rows with null values"""

reviews.dropna(axis=0,inplace=True)
reviews.isnull().sum()

Review              0
review_sentiment    0
dtype: int64

In [57]:
"""split the data into training and test sets"""

# get input feature vectors and target output
X = reviews.iloc[:, :-1]
y = reviews.iloc[:, -1:] 
# create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, shuffle=True, random_state=42)
# instantiate TfidfVectorizer sklearn
tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', min_df=5, max_features=None)
# get training and test document vectors
X_train_tfidf = tfidf.fit_transform(X_train['Review'])
X_test_tfidf = tfidf.transform(X_test['Review'])

In [58]:
"""fit multinomial Naive Bayes model to the data"""
mnb = MultinomialNB()
mnb.fit(X_train_tfidf, y_train.to_numpy().flatten())

MultinomialNB()

In [59]:
"""compute accuracy score for training set"""

mnb.score(X_train_tfidf, y_train.to_numpy().flatten())

0.7609164420485175

In [60]:
"""compute accuracy score on test data"""

mnb.score(X_test_tfidf, y_test.to_numpy().flatten())

0.7177875549968573

In [61]:
"""isntantiate sklearn pipeline and GridSearchCV"""

# create the pipeline
pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')), 
                    ('mnb', MultinomialNB())])
# specify param grid for tfidf and model
params = [{'tfidf__min_df': [1, 5, 10, 20], 'tfidf__norm': ['l1', 'l2'], 'tfidf__sublinear_tf': [True, False]}]
# instantiate and fit grid
grid_search = GridSearchCV(estimator=pipeline, param_grid=params, cv=3)
grid_search.fit(X_train['Review'], y_train.to_numpy().flatten())

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('mnb', MultinomialNB())]),
             param_grid=[{'tfidf__min_df': [1, 5, 10, 20],
                          'tfidf__norm': ['l1', 'l2'],
                          'tfidf__sublinear_tf': [True, False]}])

In [62]:
"""print the best parameters"""

grid_search.best_params_

{'tfidf__min_df': 20, 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}

In [63]:
"""make predictions on the training set"""

grid_search.score(X_train['Review'], y_train.to_numpy().flatten())

0.8018867924528302

In [64]:
"""make predictions on the test set"""

grid_search.score(X_test['Review'], y_test.to_numpy().flatten())

0.7592708988057826